Wrap-Up Script from Module 3: [Hyperparameter Tuning](https://inria.github.io/scikit-learn-mooc/tuning/parameter_tuning_wrap_up_quiz.html)

# 🏁 Wrap-up quiz 3

This quiz requires some programming to be answered.

Load the dataset file named `penguins.csv` with the following command:

In [6]:
import pandas as pd

penguins = pd.read_csv("../datasets/penguins.csv")  # adjust path if needed

columns = ["Body Mass (g)", "Flipper Length (mm)", "Culmen Length (mm)"]
target_name = "Species"

penguins_non_missing = penguins[columns + [target_name]].dropna()
X = penguins_non_missing[columns]
y = penguins_non_missing[target_name]

`penguins` is a pandas dataframe. The column “Species” contains the target variable. We extract the numerical columns that quantify some attributes of such animals and our goal is to predict their species based on those attributes stored in the dataframe named `data`.

Inspect the loaded data to select the correct assertions:

## Question 1

Inspect the target variable and select the correct assertions from the following proposals.

a) The problem to be solved is a regression problem
b) The problem to be solved is a binary classification problem (exactly 2 possible classes)
c) The problem to be solved is a multiclass classification problem (more than 2 possible classes)

Select a single answer

In [7]:
print("Number of samples: %d" % X.shape[0])
print("Number of features: %d" % X.shape[1])
print("Number of classes: %d" % y.nunique())

Number of samples: 342
Number of features: 3
Number of classes: 3


## Question 2

Inspect the statistics of the target and individual features to select the correct statements.

a) The proportions of the class counts are balanced: there are approximately the same number of rows for each class
b) The proportions of the class counts are imbalanced: some classes have more than twice as many rows than others
c) The input features have similar scales (ranges of values)

Select all answers that apply

Hint: `data.describe()`, and `target.value_counts()` are methods that are helpful to answer to this question.

In [5]:
import numpy as np

print("n_classes:", y.nunique())
print("\nclass counts:\n", y.value_counts())
print("\nclass proportions:\n", y.value_counts(normalize=True).round(3))

counts = y.value_counts()
ratio = counts.max() / counts.min()
print("\nmax/min ratio:", float(ratio))
print("imbalanced (max > 2*min)?", bool(ratio > 2))

desc = X.describe()
print("\nX.describe():\n", desc)

ranges = (desc.loc["max"] - desc.loc["min"]).rename("range")
print("\nfeature ranges:\n", ranges)
print("\nrange ratios (max_range / min_range):", float(ranges.max() / ranges.min()))

n_classes: 3

class counts:
 Species
Adelie Penguin (Pygoscelis adeliae)          151
Gentoo penguin (Pygoscelis papua)            123
Chinstrap penguin (Pygoscelis antarctica)     68
Name: count, dtype: int64

class proportions:
 Species
Adelie Penguin (Pygoscelis adeliae)          0.442
Gentoo penguin (Pygoscelis papua)            0.360
Chinstrap penguin (Pygoscelis antarctica)    0.199
Name: proportion, dtype: float64

max/min ratio: 2.2205882352941178
imbalanced (max > 2*min)? True

X.describe():
        Body Mass (g)  Flipper Length (mm)  Culmen Length (mm)
count     342.000000           342.000000          342.000000
mean     4201.754386           200.915205           43.921930
std       801.954536            14.061714            5.459584
min      2700.000000           172.000000           32.100000
25%      3550.000000           190.000000           39.225000
50%      4050.000000           197.000000           44.450000
75%      4750.000000           213.000000           48.5000

Let’s now consider the following pipeline:

In [9]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

model = Pipeline(steps=[
    ("preprocessor", StandardScaler()),
    ("classifier", KNeighborsClassifier(n_neighbors=5)),
])

## Question 3

Evaluate the pipeline using stratified 10-fold cross-validation with the `balanced-accuracy` scoring metric to choose the correct statement in the list below.

You can use:
* `sklearn.model_selection.cross_validate` to perform the cross-validation routine;
* provide an integer `10` to the parameter `cv` of `cross_validate` to use the cross-validation with 10 folds;
* provide the string `"balanced_accuracy"` to the parameter `scoring` of `cross_validate`.

a) The average cross-validated test balanced accuracy of the above pipeline is between 0.9 and 1.0
b) The average cross-validated test balanced accuracy of the above pipeline is between 0.8 and 0.9
c) The average cross-validated test balanced accuracy of the above pipeline is between 0.5 and 0.8

Select a single answer

In [11]:
from sklearn.model_selection import cross_validate

cv_5 = cross_validate(model, X, y, cv=10, scoring="balanced_accuracy", return_train_score=False)
cv_5["test_score"]

print("Cross-validation balanced accuracy scores:", cv_5["test_score"])
print("Mean balanced accuracy: %.3f" % np.mean(cv_5["test_score"]))

Cross-validation balanced accuracy scores: [1.         1.         1.         0.91880342 0.88253968 0.95238095
 0.97777778 0.93015873 0.90793651 0.95238095]
Mean balanced accuracy: 0.952


## Question 4

Repeat the evaluation by setting the parameters in order to select the correct statements in the list below. We recall that you can use `model.get_params()` to list the parameters of the pipeline and use `model.set_params(param_name=param_value)` to update them. Remember that one way to compare two models is comparing the cross-validation test scores of both models fold-to-fold, i.e. counting the number of folds where one model has a better test score than the other.

a) Looking at the individual cross-validation scores, using a model with `n_neighbors=5` is substantially better (at least 7 of the cross-validations scores are strictly better) than a model with `n_neighbors=51`
b) Looking at the individual cross-validation scores, using a model with `n_neighbors=5` is substantially better (at least 7 of the cross-validations scores are strictly better) than a model with `n_neighbors=101`
c) Looking at the individual cross-validation scores, a 5 nearest neighbors using a `StandardScaler` is substantially better (at least 7 of the cross-validations scores are strictly better) than a 5 nearest neighbors using the raw features (without scaling).

Select all answers that apply

In [12]:
from sklearn.base import clone

def cv_scores(pipeline):
    res = cross_validate(pipeline, X, y, cv=10, scoring="balanced_accuracy", return_train_score=False)
    return res["test_score"]

def fold_wins(scores_a, scores_b):
    # count folds where A strictly better than B
    return int(np.sum(scores_a > scores_b)), int(np.sum(scores_a < scores_b)), int(np.sum(scores_a == scores_b))

# n_neighbors=5 vs 51 vs 101 (with StandardScaler)
m5  = clone(model).set_params(classifier__n_neighbors=5)
m51 = clone(model).set_params(classifier__n_neighbors=51)
m101= clone(model).set_params(classifier__n_neighbors=101)

s5   = cv_scores(m5)
s51  = cv_scores(m51)
s101 = cv_scores(m101)

print("wins(5 > 51), losses, ties:", fold_wins(s5, s51))
print("wins(5 > 101), losses, ties:", fold_wins(s5, s101))

# StandardScaler vs raw features, both with n_neighbors=5
m5_raw = Pipeline(steps=[
    ("preprocessor", None),  # None means passthrough in Pipeline
    ("classifier", KNeighborsClassifier(n_neighbors=5)),
])
s5_raw = cv_scores(m5_raw)
print("wins(StdScaler+5NN > raw+5NN), losses, ties:", fold_wins(s5, s5_raw))


wins(5 > 51), losses, ties: (4, 2, 4)
wins(5 > 101), losses, ties: (10, 0, 0)
wins(StdScaler+5NN > raw+5NN), losses, ties: (10, 0, 0)


We will now study the impact of different preprocessors defined in the list below:

In [13]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import QuantileTransformer
from sklearn.preprocessing import PowerTransformer

all_preprocessors = [
    None,
    StandardScaler(),
    MinMaxScaler(),
    QuantileTransformer(n_quantiles=100),
    PowerTransformer(method="box-cox"),
]

The [Box-Cox method](https://en.wikipedia.org/wiki/Power_transform#Box%E2%80%93Cox_transformation) is common preprocessing strategy for positive values. The other preprocessors work both for any kind of numerical features. If you are curious to read the details about those method, please feel free to read them up in the [preprocessing chapter](https://scikit-learn.org/stable/modules/preprocessing.html) of the scikit-learn user guide but this is not required to answer the quiz questions.

## Question 5

Use `sklearn.model_selection.GridSearchCV` to study the impact of the choice of the preprocessor and the number of neighbors on the stratified 10-fold cross-validated `balanced_accuracy` metric. We want to study the `n_neighbors` in the range `[5, 51, 101]` and `preprocessor` in the range `all_preprocessors`. Although we wouldn’t do this in a real setting (and prefer using nested cross validation), for this question, do the cross-validation on the entire dataset.

Which of the following statements hold:

a) Looking at the individual cross-validation scores, the best ranked model using a `StandardScaler` is substantially better (at least 7 of the cross-validations scores are strictly better) than using any other preprocessor
b) Using any of the preprocessors has always a better ranking than using no preprocessor, irrespective of the value of `n_neighbors`
c) Looking at the individual cross-validation scores, the model with `n_neighbors=5` and `StandardScaler` is substantially better (at least 7 of the cross-validations scores are strictly better) than the model with `n_neighbors=51` and `StandardScaler`
d) Looking at the individual cross-validation scores, the model with `n_neighbors=51` and `StandardScaler` is substantially better (at least 7 of the cross-validations scores are strictly better) than the model with `n_neighbors=101` and `StandardScaler`

Select all answers that apply

Hint: pass `{"preprocessor": all_preprocessors, "classifier__n_neighbors": [5, 51, 101]}` for the `param_grid` argument to the `GridSearchCV` class.

In [14]:
from sklearn.model_selection import GridSearchCV

param_grid = {
    "preprocessor": all_preprocessors,
    "classifier__n_neighbors": [5, 51, 101],
}

grid = GridSearchCV(
    estimator=model,          # pipeline defined earlier
    param_grid=param_grid,
    cv=10,
    scoring="balanced_accuracy",
    n_jobs=-1,
    refit=False,              # we only need CV results
)

grid.fit(X, y)
cvres = pd.DataFrame(grid.cv_results_)
cvres[["rank_test_score", "mean_test_score", "params"]].sort_values("rank_test_score").head(10)


,rank_test_score,mean_test_score,params
1,1,0.952198,"{'classifier__n_neighbors': 5, 'preprocessor':..."
2,2,0.947778,"{'classifier__n_neighbors': 5, 'preprocessor':..."
3,3,0.947094,"{'classifier__n_neighbors': 5, 'preprocessor':..."
4,4,0.946960,"{'classifier__n_neighbors': 5, 'preprocessor':..."
6,5,0.941880,"{'classifier__n_neighbors': 51, 'preprocessor'..."
8,6,0.927277,"{'classifier__n_neighbors': 51, 'preprocessor'..."
9,7,0.922833,"{'classifier__n_neighbors': 51, 'preprocessor'..."
7,8,0.920293,"{'classifier__n_neighbors': 51, 'preprocessor'..."
11,9,0.876642,"{'classifier__n_neighbors': 101, 'preprocessor..."
12,10,0.862357,"{'classifier__n_neighbors': 101, 'preprocessor..."


## Question 6

Evaluate the generalization performance of the best models found in each fold using nested cross-validation. Set `return_estimator=True` and `cv=10` for the outer loop. The scoring metric must be the `balanced-accuracy`. The mean generalization performance is

a) better than 0.97
b) between 0.92 and 0.97
c) below 0.92

Select a single answer

In [15]:
# Utility to extract per-fold test scores for one row of cv_results_
def split_scores(row, n_splits=10):
    return np.array([row[f"split{i}_test_score"] for i in range(n_splits)])

def best_row_for(preprocessor_obj):
    mask = cvres["param_preprocessor"].apply(lambda p: p == preprocessor_obj)
    sub = cvres[mask].copy()
    # best by mean_test_score (rank_test_score is equivalent for selecting best)
    return sub.sort_values("mean_test_score", ascending=False).iloc[0]

def pretty_preproc(p):
    if p is None:
        return "None"
    return p.__class__.__name__ + (f"({getattr(p, 'method', '')})".rstrip("()") if hasattr(p, "method") else "")

# Best model using StandardScaler
best_std = best_row_for(StandardScaler())
scores_best_std = split_scores(best_std)

print("Best(StandardScaler) params:", best_std["params"], "mean:", best_std["mean_test_score"])

# (a) Best(StandardScaler) vs best of each other preprocessor
for p in all_preprocessors:
    if p is None or isinstance(p, StandardScaler):
        continue
    best_p = best_row_for(p)
    wins, losses, ties = fold_wins(scores_best_std, split_scores(best_p))
    print(f"Std best vs best {pretty_preproc(p)}: wins={wins}, losses={losses}, ties={ties}")

# (b) For each n_neighbors, check if EVERY preprocessor (excluding None) ranks better than None
for k in [5, 51, 101]:
    sub = cvres[cvres["param_classifier__n_neighbors"] == k]
    row_none = sub[sub["param_preprocessor"].isna()].iloc[0]  # None appears as NaN in this DataFrame
    rank_none = int(row_none["rank_test_score"])
    better_all = True
    for p in all_preprocessors:
        if p is None:
            continue
        row_p = sub[sub["param_preprocessor"].apply(lambda z: z == p)].iloc[0]
        if int(row_p["rank_test_score"]) >= rank_none:
            better_all = False
    print(f"n_neighbors={k}: all preprocessors rank better than None? {better_all} (rank_none={rank_none})")

# (c) & (d) StandardScaler with k=5 vs 51, and 51 vs 101 (fold-to-fold, ≥7 wins)
def row_for(preproc_obj, k):
    sub = cvres[
        cvres["param_classifier__n_neighbors"].eq(k)
        & cvres["param_preprocessor"].apply(lambda p: p == preproc_obj)
    ]
    return sub.iloc[0]

r_std_5 = row_for(StandardScaler(), 5)
r_std_51 = row_for(StandardScaler(), 51)
r_std_101 = row_for(StandardScaler(), 101)

print("StdScaler: wins(5 > 51), losses, ties:", fold_wins(split_scores(r_std_5), split_scores(r_std_51)))
print("StdScaler: wins(51 > 101), losses, ties:", fold_wins(split_scores(r_std_51), split_scores(r_std_101)))


IndexError: single positional indexer is out-of-bounds

In [16]:
from sklearn.model_selection import StratifiedKFold

inner_grid = GridSearchCV(
    estimator=model,
    param_grid=param_grid,
    scoring="balanced_accuracy",
    cv=10,
    n_jobs=-1,
)

outer = cross_validate(
    inner_grid,
    X, y,
    cv=10,
    scoring="balanced_accuracy",
    return_estimator=True,
    n_jobs=-1,
)

outer["test_score"], outer["test_score"].mean()


(array([0.95238095, 0.92673993, 1.        , 0.91880342, 0.88253968,
        1.        , 0.95555556, 0.93015873, 0.90793651, 0.95238095]),
 np.float64(0.9426495726495727))

## Question 7

Explore the set of best parameters that the different grid search models found in each fold of the outer cross-validation. Remember that you can access them with the `best_params_` attribute of the estimator. Select all the statements that are true.

a) The tuned number of nearest neighbors is stable across folds
b) The tuned number of nearest neighbors changes often across folds
c) The optimal scaler is stable across folds
d) The optimal scaler changes often across folds

Select all answers that apply

Hint: it is important to pass `return_estimator=True` to the `cross_validate` function to be able to introspect trained model saved in the `"estimator"` field of the CV results. If you forgot to do for the previous question, please re-run the cross-validation with that option enabled.

In [18]:
best_params = [est.best_params_ for est in outer["estimator"]]
best_params

[{'classifier__n_neighbors': 5,
  'preprocessor': QuantileTransformer(n_quantiles=100)},
 {'classifier__n_neighbors': 5,
  'preprocessor': QuantileTransformer(n_quantiles=100)},
 {'classifier__n_neighbors': 5, 'preprocessor': StandardScaler()},
 {'classifier__n_neighbors': 5, 'preprocessor': StandardScaler()},
 {'classifier__n_neighbors': 5, 'preprocessor': MinMaxScaler()},
 {'classifier__n_neighbors': 5,
  'preprocessor': QuantileTransformer(n_quantiles=100)},
 {'classifier__n_neighbors': 5, 'preprocessor': MinMaxScaler()},
 {'classifier__n_neighbors': 5, 'preprocessor': StandardScaler()},
 {'classifier__n_neighbors': 5, 'preprocessor': StandardScaler()},
 {'classifier__n_neighbors': 5,
  'preprocessor': QuantileTransformer(n_quantiles=100)}]

In [19]:
from collections import Counter

knn_values = [bp["classifier__n_neighbors"] for bp in best_params]
preprocs = [bp["preprocessor"] for bp in best_params]

print("n_neighbors counts:", Counter(knn_values))
print("preprocessor counts:", Counter(type(p).__name__ if p is not None else "None" for p in preprocs))


n_neighbors counts: Counter({5: 10})
preprocessor counts: Counter({'QuantileTransformer': 4, 'StandardScaler': 4, 'MinMaxScaler': 2})
